# Frequent itemsets

## Bibliotheken importeren

In [55]:
import numpy as np
import pandas as pd
import pyodbc
import sqlite3
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth


## Data inlezen en kolommen selecteren

**connectie**

In [56]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor 

Adventure = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")


<>:1: DeprecationWarning: invalid escape sequence '\S'
<>:11: DeprecationWarning: invalid escape sequence '\S'
<>:17: DeprecationWarning: invalid escape sequence '\S'
<>:1: DeprecationWarning: invalid escape sequence '\S'
<>:11: DeprecationWarning: invalid escape sequence '\S'
<>:17: DeprecationWarning: invalid escape sequence '\S'
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1073580416.py:1: DeprecationWarning: invalid escape sequence '\S'
  DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS',
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1073580416.py:11: DeprecationWarning: invalid escape sequence '\S'
  'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1073580416.py:17: DeprecationWarning: invalid escape sequence '\S'
  'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',


**Dataframes inlezen**

In [57]:
AdventureWorks_queries = {
    'SalesOrderHeader' : 'SELECT * FROM Sales.SalesOrderHeader',
    'SalesOrderDetail' : 'SELECT * FROM Sales.SalesOrderDetail'
}

Northwind_queries = {
    'Orders' : 'SELECT * FROM Orders',
    'OrderDetails' : 'SELECT * FROM [Order Details]'
}

AenC_queries = {
    'sales_order' : 'SELECT * FROM sales_order',
    'sales_order_item' : 'SELECT * FROM sales_order_item',
    'Product' : 'SELECT * FROM Product' 
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1319449748.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1319449748.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1319449748.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table

### Product

In [58]:
products_query = """
SELECT NULL AS NewProductID, P.ProductID, P.ProductName, C.Description AS ProductDescription, C.CategoryName AS ProductCategory, 
       NULL AS Color, NULL AS Size, NULL AS ProductQuantity, P.QuantityPerUnit, CAST(P.UnitPrice AS FLOAT) AS UnitPrice, NULL AS ListPrice, P.UnitsInStock 
FROM Products P 
JOIN Categories C ON P.CategoryID = C.CategoryID
"""
products_df = pd.read_sql_query(products_query, Northwind_conn)
products_df['Source'] = 'Northwind'

products_df['ProductID'] = products_df['ProductID'].apply(lambda x: f'NW_{x}')

print(products_df)

   NewProductID ProductID                      ProductName  \
0          None      NW_1                             Chai   
1          None      NW_2                            Chang   
2          None      NW_3                    Aniseed Syrup   
3          None      NW_4     Chef Anton's Cajun Seasoning   
4          None      NW_5           Chef Anton's Gumbo Mix   
..          ...       ...                              ...   
72         None     NW_73                       Röd Kaviar   
73         None     NW_74                    Longlife Tofu   
74         None     NW_75             Rhönbräu Klosterbier   
75         None     NW_76                     Lakkalikööri   
76         None     NW_77  Original Frankfurter grüne Soße   

                                   ProductDescription ProductCategory Color  \
0         Soft drinks, coffees, teas, beers, and ales       Beverages  None   
1         Soft drinks, coffees, teas, beers, and ales       Beverages  None   
2   Sweet and savo

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\3906597526.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products_df = pd.read_sql_query(products_query, Northwind_conn)


In [59]:
product_query = """
SELECT NULL AS NewProductID, id AS ProductID, name AS ProductName, description AS ProductDescription, Category AS ProductCategory, 
       color AS ProductColor, prod_size AS ProductSize, quantity AS ProductQuantity, NULL AS QuantityPerUnit, unit_price AS UnitPrice, 
       NULL AS ListPrice, NULL AS UnitsInStock
FROM product
"""
product_df = pd.read_sql_query(product_query, AenC_conn)
product_df['Source'] = 'AenC'

product_df['ProductID'] = product_df['ProductID'].apply(lambda x: f'AC_{x}')

# Converteer UnitPrice kolom naar float
product_df['UnitPrice'] = product_df['UnitPrice'].astype(float)
print(product_df)

  NewProductID ProductID   ProductName ProductDescription ProductCategory  \
0         None    AC_300     Tee Shirt           Tank Top         Clothes   
1         None    AC_301     Tee Shirt             V-neck         Clothes   
2         None    AC_302     Tee Shirt          Crew Neck         Clothes   
3         None    AC_400  Baseball Cap         Cotton Cap     Accessories   
4         None    AC_401  Baseball Cap           Wool cap     Accessories   
5         None    AC_500         Visor        Cloth Visor     Accessories   
6         None    AC_501         Visor      Plastic Visor     Accessories   
7         None    AC_600    Sweatshirt  Hooded Sweatshirt         Clothes   
8         None    AC_601    Sweatshirt  Zipped Sweatshirt         Clothes   
9         None    AC_700        Shorts      Cotton Shorts         Clothes   

  ProductColor        ProductSize ProductQuantity QuantityPerUnit  UnitPrice  \
0        White              Small              18            None       

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\2147365399.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_df = pd.read_sql_query(product_query, AenC_conn)


In [60]:
new_product_query = """
SELECT NULL AS NewProductID, P.ProductID, P.Name AS ProductName, S.Name AS ProductDescription, C.Name AS ProductCategory, P.Color AS ProductColor, P.Size AS ProductSize, NULL AS ProductQuantity, 
       NULL AS QuantityPerUnit, CAST(P.StandardCost AS FLOAT) AS UnitPrice, CAST(P.ListPrice AS FLOAT) AS ListPrice, NULL AS UnitsInStock
FROM Production.Product P
JOIN Production.ProductSubcategory S
ON p.ProductSubcategoryID = S.ProductSubcategoryID
JOIN Production.ProductCategory C 
ON S.ProductCategoryID = C.ProductCategoryID
"""
new_product_df = pd.read_sql_query(new_product_query, Adventure_conn)
new_product_df['Source'] = 'AdventureWorks'

new_product_df['ProductID'] = new_product_df['ProductID'].apply(lambda x: f'AW_{x}')


print(new_product_df)

    NewProductID ProductID                ProductName ProductDescription  \
0           None    AW_680  HL Road Frame - Black, 58        Road Frames   
1           None    AW_706    HL Road Frame - Red, 58        Road Frames   
2           None    AW_707      Sport-100 Helmet, Red            Helmets   
3           None    AW_708    Sport-100 Helmet, Black            Helmets   
4           None    AW_709     Mountain Bike Socks, M              Socks   
..           ...       ...                        ...                ...   
290         None    AW_995          ML Bottom Bracket    Bottom Brackets   
291         None    AW_996          HL Bottom Bracket    Bottom Brackets   
292         None    AW_997         Road-750 Black, 44         Road Bikes   
293         None    AW_998         Road-750 Black, 48         Road Bikes   
294         None    AW_999         Road-750 Black, 52         Road Bikes   

    ProductCategory ProductColor ProductSize ProductQuantity QuantityPerUnit  \
0      

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\565520136.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_product_df = pd.read_sql_query(new_product_query, Adventure_conn)


In [61]:
# Voeg de ontbrekende kolommen toe aan elke DataFrame en orden ze in dezelfde volgorde
columns = ['NewProductID', 'ProductID', 'Source', 'ProductName', 'ProductDescription', 'ProductCategory', 'ProductColor', 'ProductSize', 'ProductQuantity', 
           'QuantityPerUnit', 'UnitPrice', 'ListPrice', 'UnitsInStock']

products_df = products_df.reindex(columns=columns)
product_df = product_df.reindex(columns=columns)
new_product_df = new_product_df.reindex(columns=columns)

# Combineer de dataframes
combined_df = pd.concat([products_df, product_df, new_product_df], ignore_index=True)

# Genereer een nieuwe primaire sleutel
combined_df['NewProductID'] = range(1, len(combined_df) + 1)

# Converteer null waarden naar 0.0 voor UnitPrice en ListPrice
combined_df['UnitPrice'] = combined_df['UnitPrice'].fillna(0.0).astype(float)
combined_df['ListPrice'] = combined_df['ListPrice'].fillna(0.0).astype(float)

# Zorg ervoor dat alle waarden correct zijn getypt
combined_df['NewProductID'] = combined_df['NewProductID'].astype(int)
combined_df['ProductID'] = combined_df['ProductID']
combined_df['Source'] = combined_df['Source'].astype(str)
combined_df['ProductName'] = combined_df['ProductName'].astype(str)
combined_df['ProductDescription'] = combined_df['ProductDescription'].astype(str)
combined_df['ProductCategory'] = combined_df['ProductCategory'].astype(str)
combined_df['ProductColor'] = combined_df['ProductColor'].astype(str)
combined_df['ProductSize'] = combined_df['ProductSize'].astype(str)
combined_df['ProductQuantity'] = combined_df['ProductQuantity'].fillna(0).astype(int)
combined_df['QuantityPerUnit'] = combined_df['QuantityPerUnit'].astype(str)
combined_df['UnitsInStock'] = combined_df['UnitsInStock'].fillna(0).astype(int)


print(combined_df)

     NewProductID ProductID          Source                   ProductName  \
0               1      NW_1       Northwind                          Chai   
1               2      NW_2       Northwind                         Chang   
2               3      NW_3       Northwind                 Aniseed Syrup   
3               4      NW_4       Northwind  Chef Anton's Cajun Seasoning   
4               5      NW_5       Northwind        Chef Anton's Gumbo Mix   
..            ...       ...             ...                           ...   
377           378    AW_995  AdventureWorks             ML Bottom Bracket   
378           379    AW_996  AdventureWorks             HL Bottom Bracket   
379           380    AW_997  AdventureWorks            Road-750 Black, 44   
380           381    AW_998  AdventureWorks            Road-750 Black, 48   
381           382    AW_999  AdventureWorks            Road-750 Black, 52   

                                    ProductDescription ProductCategory  \
0

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\2012267552.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([products_df, product_df, new_product_df], ignore_index=True)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\2012267552.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df['UnitsInStock'] = combined_df['UnitsInStock'].fillna(0).astype(int)


### Order

In [62]:
#eerst horizontaal samenvoegen per database:
AdventureWorks_Order = pd.merge(SalesOrderHeader, SalesOrderDetail, on = 'SalesOrderID')
Northwind_Order = pd.merge(Orders, OrderDetails, on = 'OrderID')
merge1 = pd.merge(Product, sales_order_item, left_on= 'id', right_on= 'prod_id')
AenC_Order = pd.merge(merge1, sales_order, left_on = 'id_y', right_on = 'id')

print(AdventureWorks_Order.dtypes)

SalesOrderID                       int64
RevisionNumber                     int64
OrderDate                 datetime64[ns]
DueDate                   datetime64[ns]
ShipDate                  datetime64[ns]
Status                             int64
OnlineOrderFlag                     bool
SalesOrderNumber                  object
PurchaseOrderNumber               object
AccountNumber                     object
CustomerID                         int64
SalesPersonID                    float64
TerritoryID                        int64
BillToAddressID                    int64
ShipToAddressID                    int64
ShipMethodID                       int64
CreditCardID                     float64
CreditCardApprovalCode            object
CurrencyRateID                   float64
SubTotal                         float64
TaxAmt                           float64
Freight                          float64
TotalDue                         float64
Comment                           object
rowguid_x       

**Transform**

In [63]:
AdventureWorks_Order.rename(columns={
    'SalesOrderID': 'OrderID',
    'SalesOrderDetailID': 'OrderDetailID',
    'OrderQty': 'Quantity',
    'UnitPriceDiscount': 'Discount',
    'Order_Date': 'OrderDate',
    'SalesPersonID' : 'EmployeeID'
}, inplace=True)

# Vul NaN waarden in EmployeeID met een standaardwaarde (bijv. 0)
AdventureWorks_Order['EmployeeID'].fillna(0, inplace=True)

# change employe id type to int

AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].astype(int)

# change shipvia to string 
#Northwind_Order['ShipVia'] = Northwind_Order['ShipVia'].astype(str)
Northwind_Order.rename(columns={
    'ShipVia': 'ShipMethodID'
}, inplace=True)

AenC_Order.rename(columns={
    'id': 'OrderID',
    'item_id': 'OrderDetailID',
    'quantity_y': 'Quantity',
    'order_date': 'OrderDate',
    'unit_price': 'UnitPrice',
    'cust_id': 'CustomerID',
    'sales_rep': 'EmployeeID',
    'prod_id': 'ProductID'
}, inplace=True)

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\2543201151.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AdventureWorks_Order['EmployeeID'].fillna(0, inplace=True)


**Alle dubbele ID's met een suffix zetten per database**

In [64]:
# Prefixes toevoegen
AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].apply(lambda x: f'AW_{x}')
Northwind_Order['EmployeeID'] = Northwind_Order['EmployeeID'].apply(lambda x: f'NW_{x}')
AenC_Order['EmployeeID'] = AenC_Order['EmployeeID'].apply(lambda x: f'AC_{x}')

# Vervang 'AW_0' met 'NULL' in de kolom EmployeeID
AdventureWorks_Order['EmployeeID'] = AdventureWorks_Order['EmployeeID'].replace('AW_0', None)

AdventureWorks_Order['ShipMethodID'] = AdventureWorks_Order['ShipMethodID'].apply(lambda x: f'AW_{x}')
Northwind_Order['ShipMethodID'] = Northwind_Order['ShipMethodID'].apply(lambda x: f'NW_{x}')
# AenC_Order heeft geen ShipMethodID

AdventureWorks_Order['ProductID'] = AdventureWorks_Order['ProductID'].apply(lambda x: f'AW_{x}')
Northwind_Order['ProductID'] = Northwind_Order['ProductID'].apply(lambda x: f'NW_{x}')
AenC_Order['ProductID'] = AenC_Order['ProductID'].apply(lambda x: f'AC_{x}')

AdventureWorks_Order['CustomerID'] = AdventureWorks_Order['CustomerID'].apply(lambda x: f'AW_{x}')
Northwind_Order['CustomerID'] = Northwind_Order['CustomerID'].apply(lambda x: f'NW_{x}')
AenC_Order['CustomerID'] = AenC_Order['CustomerID'].apply(lambda x: f'AC_{x}')

Order dataframe aanmaken

In [65]:
# Zorg ervoor dat alle vereiste kolommen bestaan en selecteer ze
required_columns = ['OrderID', 'OrderDetailID', 'Quantity', 'UnitPrice', 'Discount', 'OrderDate', 'CustomerID', 'EmployeeID', 'ShipMethodID', 'ProductID']

for col in required_columns:
    if col not in AdventureWorks_Order.columns:
        AdventureWorks_Order[col] = None
    if col not in Northwind_Order.columns:
        Northwind_Order[col] = None
    if col not in AenC_Order.columns:
        AenC_Order[col] = None

# Selecteer de kolommen in de juiste volgorde
AdventureWorks_Order = AdventureWorks_Order[required_columns]
Northwind_Order = Northwind_Order[required_columns]
AenC_Order = AenC_Order[required_columns]

# Combineer de dataframes
Order = pd.concat([AdventureWorks_Order, Northwind_Order, AenC_Order], ignore_index=True)

# Converteer 'Quantity' en 'UnitPrice' naar numerieke typen
Order['Quantity'] = pd.to_numeric(Order['Quantity'], errors='coerce')
Order['UnitPrice'] = pd.to_numeric(Order['UnitPrice'], errors='coerce')

Order['Omzet'] = Order['UnitPrice'] * Order['Quantity']

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\2759866656.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Order = pd.concat([AdventureWorks_Order, Northwind_Order, AenC_Order], ignore_index=True)


**Kolomen selecteren**

In [66]:
Order

,OrderID,OrderDetailID,Quantity,UnitPrice,Discount,OrderDate,CustomerID,EmployeeID,ShipMethodID,ProductID,Omzet
0,43659,1,1,2024.994,0.0,2011-05-31 00:00:00,AW_29825,AW_279,AW_5,AW_776,2024.994
1,43659,2,3,2024.994,0.0,2011-05-31 00:00:00,AW_29825,AW_279,AW_5,AW_777,6074.982
2,43659,3,1,2024.994,0.0,2011-05-31 00:00:00,AW_29825,AW_279,AW_5,AW_778,2024.994
3,43659,4,1,2039.994,0.0,2011-05-31 00:00:00,AW_29825,AW_279,AW_5,AW_771,2039.994
4,43659,5,1,2039.994,0.0,2011-05-31 00:00:00,AW_29825,AW_279,AW_5,AW_772,2039.994
...,...,...,...,...,...,...,...,...,...,...,...
124570,2625,None,24,15.000,NaN,08-Nov-1998 12:00:00 AM,AC_122,AC_690,None,AC_700,360.000
124571,2635,None,72,15.000,NaN,15-Nov-1998 12:00:00 AM,AC_127,AC_129,None,AC_700,1080.000
124572,2641,None,24,15.000,NaN,18-Nov-1998 12:00:00 AM,AC_130,AC_949,None,AC_700,360.000
124573,2649,None,36,15.000,NaN,25-Nov-1998 12:00:00 AM,AC_132,AC_690,None,AC_700,540.000


In [67]:
Order.columns


Index(['OrderID', 'OrderDetailID', 'Quantity', 'UnitPrice', 'Discount',
       'OrderDate', 'CustomerID', 'EmployeeID', 'ShipMethodID', 'ProductID',
       'Omzet'],
      dtype='object')

**Alle rijen in df moeten een transactie voortstellen**

In [68]:
Order = Order.loc[:, ['OrderID', 'ProductID']]
Order

,OrderID,ProductID
0,43659,AW_776
1,43659,AW_777
2,43659,AW_778
3,43659,AW_771
4,43659,AW_772
...,...,...
124570,2625,AC_700
124571,2635,AC_700
124572,2641,AC_700
124573,2649,AC_700


## Dummy encoding

**verzameling itemsets**

In [69]:
df_dummies = pd.get_dummies(Order, columns=['ProductID'], prefix='', prefix_sep='')

#de max() functie kijkt of een product aanwezig is in een order at least once
#dit doet het door de max value te nemen voor elk entry voor een product in een order
df_dummies = df_dummies.groupby(Order['OrderID']).max() 
df_dummies

,OrderID,AC_300,AC_301,AC_302,AC_400,AC_401,AC_500,AC_501,AC_600,AC_601,...,NW_70,NW_71,NW_72,NW_73,NW_74,NW_75,NW_76,NW_77,NW_8,NW_9
OrderID,,,,,,,,,,,,,,,,,,,,,
10248,10248,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
10249,10249,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10250,10250,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10251,10251,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10252,10252,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,2650,False,False,False,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2651,2651,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2652,2652,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Apriori algoritme toepassen

**Bereken de frequent itemsets door de itemsets te selecteren >= min_support**

In [70]:
df_dummies = df_dummies.drop(['OrderID'], axis = 1)

#vind frequente itemsets
frequent_itemsets = apriori(df_dummies, min_support= 0.02, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.093580,(AW_707)
1,0.091273,(AW_708)
2,0.093793,(AW_711)
3,0.102656,(AW_712)
4,0.036971,(AW_714)
...,...,...
88,0.022492,"(AW_707, AW_715, AW_711, AW_708)"
89,0.022371,"(AW_707, AW_712, AW_715, AW_708)"
90,0.022310,"(AW_707, AW_712, AW_715, AW_711)"
91,0.023706,"(AW_712, AW_715, AW_711, AW_708)"


**Rules Generation met low confidence rule** 

In [71]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AW_707),(AW_708),0.093580,0.091273,0.024040,0.256893,2.814542,0.015499,1.222874,0.711263
1,(AW_708),(AW_707),0.091273,0.093580,0.024040,0.263385,2.814542,0.015499,1.230521,0.709457
2,(AW_707),(AW_711),0.093580,0.093793,0.024070,0.257217,2.742399,0.015293,1.220016,0.700951
3,(AW_711),(AW_707),0.093793,0.093580,0.024070,0.256634,2.742399,0.015293,1.219346,0.701115
4,(AW_707),(AW_712),0.093580,0.102656,0.028988,0.309763,3.017489,0.019381,1.300053,0.737626
...,...,...,...,...,...,...,...,...,...,...
211,(AW_707),"(AW_712, AW_715, AW_711, AW_708)",0.093580,0.023706,0.021733,0.232241,9.796659,0.019515,1.271615,0.990628
212,(AW_711),"(AW_707, AW_712, AW_715, AW_708)",0.093793,0.022371,0.021733,0.231715,10.358016,0.019635,1.272483,0.996964
213,(AW_715),"(AW_707, AW_712, AW_711, AW_708)",0.049628,0.022280,0.021733,0.437920,19.655709,0.020627,1.739470,0.998687
214,(AW_712),"(AW_707, AW_715, AW_711, AW_708)",0.102656,0.022492,0.021733,0.211709,9.412624,0.019424,1.240034,0.996006


**Strong Rules identificeren om te kijken welke producten het meest samen worden gekocht**

In [72]:
# Sorteer deze regels op basis van confidence, lift en support
rules = rules.sort_values(['confidence', 'lift', 'support'], ascending=[False, False, False])

# Bekijk de top regels om te zien welke producten vaak samen gekocht worden
top_rules = rules.head(20)
print(top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                          antecedents consequents   support  confidence  \
188  (AW_707, AW_712, AW_711, AW_708)    (AW_715)  0.021733    0.975477   
186  (AW_707, AW_712, AW_715, AW_711)    (AW_708)  0.021733    0.974150   
145          (AW_707, AW_712, AW_708)    (AW_715)  0.022371    0.973580   
159          (AW_707, AW_712, AW_711)    (AW_715)  0.022310    0.973510   
116          (AW_707, AW_712, AW_711)    (AW_708)  0.022280    0.972185   
189  (AW_707, AW_712, AW_715, AW_708)    (AW_711)  0.021733    0.971506   
174          (AW_712, AW_711, AW_708)    (AW_715)  0.023706    0.971393   
131          (AW_707, AW_715, AW_708)    (AW_711)  0.022492    0.971166   
117          (AW_707, AW_712, AW_708)    (AW_711)  0.022280    0.969617   
132          (AW_707, AW_711, AW_708)    (AW_715)  0.022492    0.968627   
130          (AW_707, AW_715, AW_711)    (AW_708)  0.022492    0.968627   
175          (AW_715, AW_711, AW_708)    (AW_712)  0.023706    0.967782   
144          (AW_707, AW_

**product nummers omzetten naar namen, voor een duidelijker overzicht**

In [73]:
product_name_mapping = pd.Series(combined_df.ProductName.values, index=combined_df.ProductID).to_dict()

# Nu gaan we door de regels DataFrame heen en vertalen de antecedents en consequents
def get_product_names(itemset):
    return [product_name_mapping[item] for item in itemset]


# Pas deze functie toe om de productnamen te krijgen
rules['antecedent_names'] = rules['antecedents'].apply(get_product_names)
rules['consequent_names'] = rules['consequents'].apply(get_product_names)

# Maak een nieuw DataFrame met de gewenste informatie
new_df = rules[['antecedent_names', 'consequent_names', 'confidence']]
# Converteer confidence naar een afgerond percentage en voeg een '%' toe
new_df['confidence'] = (new_df['confidence'] * 100).round().astype(int).astype(str) + '%'

new_df.head(20)


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\2906392046.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['confidence'] = (new_df['confidence'] * 100).round().astype(int).astype(str) + '%'


,antecedent_names,consequent_names,confidence
188,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Long-Sleeve Logo Jersey, L]",98%
186,"[Sport-100 Helmet, Red, AWC Logo Cap, Long-Sle...","[Sport-100 Helmet, Black]",97%
145,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Long-Sleeve Logo Jersey, L]",97%
159,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Long-Sleeve Logo Jersey, L]",97%
116,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Sport-100 Helmet, Black]",97%
189,"[Sport-100 Helmet, Red, AWC Logo Cap, Long-Sle...","[Sport-100 Helmet, Blue]",97%
174,"[AWC Logo Cap, Sport-100 Helmet, Blue, Sport-1...","[Long-Sleeve Logo Jersey, L]",97%
131,"[Sport-100 Helmet, Red, Long-Sleeve Logo Jerse...","[Sport-100 Helmet, Blue]",97%
117,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Sport-100 Helmet, Blue]",97%
132,"[Sport-100 Helmet, Red, Sport-100 Helmet, Blue...","[Long-Sleeve Logo Jersey, L]",97%


## Apriori algoritme met closed en maximale itemsets

In [74]:
frequent_itemsets

,support,itemsets
0,0.093580,(AW_707)
1,0.091273,(AW_708)
2,0.093793,(AW_711)
3,0.102656,(AW_712)
4,0.036971,(AW_714)
...,...,...
88,0.022492,"(AW_707, AW_715, AW_711, AW_708)"
89,0.022371,"(AW_707, AW_712, AW_715, AW_708)"
90,0.022310,"(AW_707, AW_712, AW_715, AW_711)"
91,0.023706,"(AW_712, AW_715, AW_711, AW_708)"


**Filter op closed itemsets**

**Closed itemsets:** Een itemset is gesloten als er geen superset van bestaat die dezelfde support heeft. 

**Support** is hoevaak komt een rij(itemsets) voor in totale verzameling itemsets \
met closed itemsets filter je sets weg gebaseerd of een superset dezelfde support heeft

In [75]:
def find_closed_itemsets(frequent_itemsets):
    # Voeg een kolom toe voor het bijhouden van closed status
    frequent_itemsets['is_closed'] = True

    # Converteer set voor efficiënte subset checks
    # Dictionary comprehension, frozenset om de itemsets als sleutel te gebruiken.
    # Itereer over itemset en support kolom en zip() lever tuple op met huidige itemset en support
    itemset_support_dict = {frozenset(itemset): support for itemset, support in zip(frequent_itemsets['itemsets'], frequent_itemsets['support'])}
    
    # Controleer op elke itemset of het closed is
    for itemset in itemset_support_dict:
        current_support = itemset_support_dict[itemset]
        # Check elke andere itemset om te zien of het een superset is met dezelfde support
        for potential_superset in itemset_support_dict:
            if itemset < potential_superset and itemset_support_dict[potential_superset] == current_support:
                # Als een superset met dezelfde support bestaat, is deze itemset niet closed
                frequent_itemsets.loc[frequent_itemsets['itemsets'] == itemset, 'is_closed'] = False
                break

    # Filter om alleen closed itemsets te behouden
    closed_itemsets = frequent_itemsets[frequent_itemsets['is_closed']].drop('is_closed', axis=1)
    return closed_itemsets

# Vind closed itemsets
closed_itemsets = find_closed_itemsets(frequent_itemsets)
closed_itemsets

,support,itemsets
0,0.093580,(AW_707)
1,0.091273,(AW_708)
2,0.093793,(AW_711)
3,0.102656,(AW_712)
4,0.036971,(AW_714)
...,...,...
88,0.022492,"(AW_707, AW_715, AW_711, AW_708)"
89,0.022371,"(AW_707, AW_712, AW_715, AW_708)"
90,0.022310,"(AW_707, AW_712, AW_715, AW_711)"
91,0.023706,"(AW_712, AW_715, AW_711, AW_708)"


**Filter op maximale itemsets**

**Maximale itemsets:** Een itemset is maximaal als er geen frequente superset van bestaat.

In [76]:
def find_maximal_itemsets(closed_itemsets):
    itemset_support_dict = {frozenset(itemset): support for itemset, support in zip(closed_itemsets['itemsets'], closed_itemsets['support'])}
    maximal_itemsets = []
    
    # Controleer elk itemset
    for itemset in itemset_support_dict.keys():
        is_maximal = True
        current_support = itemset_support_dict[itemset]
        
        # Controleer elk ander itemset
        for potential_superset in itemset_support_dict.keys():
            # Een itemset kan alleen maximaal zijn als er geen grotere frequente superset bestaat
            if itemset < potential_superset and itemset_support_dict[potential_superset] >= current_support:
                is_maximal = False
                break
        
        if is_maximal:
            maximal_itemsets.append((itemset, current_support))
    
    return maximal_itemsets

# Roep de functie aan om maximale itemsets te vinden
maximal_itemsets = find_maximal_itemsets(closed_itemsets)
maximal_df = pd.DataFrame(maximal_itemsets, columns=['itemsets', 'support'])
maximal_df

,itemsets,support
0,(AW_707),0.093580
1,(AW_708),0.091273
2,(AW_711),0.093793
3,(AW_712),0.102656
4,(AW_714),0.036971
...,...,...
88,"(AW_707, AW_715, AW_711, AW_708)",0.022492
89,"(AW_707, AW_712, AW_715, AW_708)",0.022371
90,"(AW_707, AW_712, AW_715, AW_711)",0.022310
91,"(AW_712, AW_715, AW_711, AW_708)",0.023706


**Rules Generation met low confidence rule** 

In [77]:
rules = association_rules(maximal_df, metric="confidence", min_threshold=0.1)
rules 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AW_707),(AW_708),0.093580,0.091273,0.024040,0.256893,2.814542,0.015499,1.222874,0.711263
1,(AW_708),(AW_707),0.091273,0.093580,0.024040,0.263385,2.814542,0.015499,1.230521,0.709457
2,(AW_707),(AW_711),0.093580,0.093793,0.024070,0.257217,2.742399,0.015293,1.220016,0.700951
3,(AW_711),(AW_707),0.093793,0.093580,0.024070,0.256634,2.742399,0.015293,1.219346,0.701115
4,(AW_707),(AW_712),0.093580,0.102656,0.028988,0.309763,3.017489,0.019381,1.300053,0.737626
...,...,...,...,...,...,...,...,...,...,...
211,(AW_707),"(AW_712, AW_715, AW_711, AW_708)",0.093580,0.023706,0.021733,0.232241,9.796659,0.019515,1.271615,0.990628
212,(AW_711),"(AW_707, AW_712, AW_715, AW_708)",0.093793,0.022371,0.021733,0.231715,10.358016,0.019635,1.272483,0.996964
213,(AW_715),"(AW_707, AW_712, AW_711, AW_708)",0.049628,0.022280,0.021733,0.437920,19.655709,0.020627,1.739470,0.998687
214,(AW_712),"(AW_707, AW_715, AW_711, AW_708)",0.102656,0.022492,0.021733,0.211709,9.412624,0.019424,1.240034,0.996006


**Strong Rules identificeren om te kijken welke producten het meest samen worden gekocht**

In [78]:
# Sorteer deze regels op basis van confidence, lift en support
rules = rules.sort_values(['confidence', 'lift', 'support'], ascending=[False, False, False])

# Bekijk de top regels om te zien welke producten vaak samen gekocht worden
top_rules = rules.head(20)
print(top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                          antecedents consequents   support  confidence  \
188  (AW_707, AW_712, AW_711, AW_708)    (AW_715)  0.021733    0.975477   
186  (AW_707, AW_712, AW_715, AW_711)    (AW_708)  0.021733    0.974150   
145          (AW_707, AW_712, AW_708)    (AW_715)  0.022371    0.973580   
159          (AW_707, AW_712, AW_711)    (AW_715)  0.022310    0.973510   
116          (AW_707, AW_712, AW_711)    (AW_708)  0.022280    0.972185   
189  (AW_707, AW_712, AW_715, AW_708)    (AW_711)  0.021733    0.971506   
174          (AW_712, AW_711, AW_708)    (AW_715)  0.023706    0.971393   
131          (AW_707, AW_715, AW_708)    (AW_711)  0.022492    0.971166   
117          (AW_707, AW_712, AW_708)    (AW_711)  0.022280    0.969617   
132          (AW_707, AW_711, AW_708)    (AW_715)  0.022492    0.968627   
130          (AW_707, AW_715, AW_711)    (AW_708)  0.022492    0.968627   
175          (AW_715, AW_711, AW_708)    (AW_712)  0.023706    0.967782   
144          (AW_707, AW_

**product nummers omzetten naar namen, voor een duidelijker overzicht**

In [79]:
product_name_mapping = pd.Series(combined_df.ProductName.values, index=combined_df.ProductID).to_dict()

# Nu gaan we door de regels DataFrame heen en vertalen de antecedents en consequents
def get_product_names(itemset):
    return [product_name_mapping[item] for item in itemset]


# Pas deze functie toe om de productnamen te krijgen
rules['antecedent_names'] = rules['antecedents'].apply(get_product_names)
rules['consequent_names'] = rules['consequents'].apply(get_product_names)

# Maak een nieuw DataFrame met de gewenste informatie
new_df = rules[['antecedent_names', 'consequent_names', 'confidence']]
# Converteer confidence naar een afgerond percentage en voeg een '%' toe
new_df['confidence'] = (new_df['confidence'] * 100).round().astype(int).astype(str) + '%'

new_df.head(20)

C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_20064\1842551028.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['confidence'] = (new_df['confidence'] * 100).round().astype(int).astype(str) + '%'


,antecedent_names,consequent_names,confidence
188,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Long-Sleeve Logo Jersey, L]",98%
186,"[Sport-100 Helmet, Red, AWC Logo Cap, Long-Sle...","[Sport-100 Helmet, Black]",97%
145,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Long-Sleeve Logo Jersey, L]",97%
159,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Long-Sleeve Logo Jersey, L]",97%
116,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Sport-100 Helmet, Black]",97%
189,"[Sport-100 Helmet, Red, AWC Logo Cap, Long-Sle...","[Sport-100 Helmet, Blue]",97%
174,"[AWC Logo Cap, Sport-100 Helmet, Blue, Sport-1...","[Long-Sleeve Logo Jersey, L]",97%
131,"[Sport-100 Helmet, Red, Long-Sleeve Logo Jerse...","[Sport-100 Helmet, Blue]",97%
117,"[Sport-100 Helmet, Red, AWC Logo Cap, Sport-10...","[Sport-100 Helmet, Blue]",97%
132,"[Sport-100 Helmet, Red, Sport-100 Helmet, Blue...","[Long-Sleeve Logo Jersey, L]",97%
